# AutoCAD MegaIntellegence Project
  Предназначено для ускорения работы с конструкторскими чертежами. Машинное обучение позволит убрать рутинные операции. Программа должна проставлять размеры на точки крепления к приборам, с учётом всех требований. Это первая задача. С имеющейся базой можно расширить применение на другие задачи работы с dwg чертежами.
  
  Проект читает $хорошие$ чертежи, обучается на существующих позициях размеров. X - линии в слое с точками крепления, пересечение с линиями прибора. y - базовые точки размеров, положение текста
  
  Первая версия: будет один вид, нужно проставить все размеры на 1 виде.
  
$$успех = усердие + время$$

In [1]:

import win32com.client
import comtypes.client
import numpy as np
import pandas as pd
import math
import array

app = comtypes.client.GetActiveObject('AutoCAD.Application')
#app = win32com.client.Dispatch("AutoCAD.Application")
activedoc = app.ActiveDocument
msp =activedoc.ModelSpace

In [2]:
# import win32com.client
# acad = win32com.client.Dispatch("AutoCAD.Application")

# doc = acad.ActiveDocument   # Document object


# # iterate trough all objects (entities) in the currently opened drawing
# # and if its a BlockReference, display its attributes and some other things.
# for entity in acad.ActiveDocument.ModelSpace:
#     name = entity.EntityName
#     if name == 'AcDbBlockReference':
#         HasAttributes = entity.HasAttributes
#         if HasAttributes:
#             print(entity.Name)
#             print(entity.Layer)
#             print(entity.ObjectID)
#             for attrib in entity.GetAttributes():
#                 print("  {}: {}".format(attrib.TagString, attrib.TextString))
                
#                 # update text
#                 attrib.TextString = 'modified with python'
#                 attrib.Update()

In [10]:
for blk in activedoc.Blocks:
    print(blk)
#    for ent in blk:
#         print(ent)

<POINTER(IAcadModelSpace) ptr=0x72b42e8 at 94943c8>
<POINTER(IAcadPaperSpace) ptr=0x72c0468 at 5276148>
<POINTER(IAcadBlock) ptr=0x771a478 at 948e2c8>
<POINTER(IAcadBlock) ptr=0x72c0318 at 5276148>
<POINTER(IAcadBlock) ptr=0x771acc8 at 948e7c8>
<POINTER(IAcadBlock) ptr=0x72bfc88 at 5276148>
<POINTER(IAcadBlock) ptr=0x771a9b8 at 948eec8>
<POINTER(IAcadBlock) ptr=0x72c0548 at 5276148>


In [115]:
# set variables
df = pd.DataFrame(columns ={'device','start_att_line_X', 'end_att_line_X','start_att_line_Y', 'end_att_line_Y'})
df = df[['device','start_att_line_X', 'end_att_line_X','start_att_line_Y', 'end_att_line_Y']]
dim_aligned, dim_rotated, blockref, i,bl = 0,0,0,0,0
# view each object
for item in msp:
    #TODO: need to check if that actually device block
    #if it's a device:
    if item.ObjectName == 'AcDbBlockReference':
        #find BLOCK object by given BlockRef
        block = activedoc.Blocks.Item(item.Name)
        bl=item
        print (f'Block name: {block}')
        for element in block:
            if element.Layer =="attachment point":
                # fill the dataframe with the coordinates of the attachment points
                  df = df.append({'device': i, \
                                'start_att_line_X': element.StartPoint[0] + item.InsertionPoint[0], \
                                'start_att_line_Y': element.StartPoint[1] + item.InsertionPoint[1], \
                                'end_att_line_X': element.EndPoint[0] + item.InsertionPoint[0],\
                                'end_att_line_Y': element.EndPoint[1] + item.InsertionPoint[1]},  ignore_index=True)
        i+=1
                
df['device'] = df['device'].astype(int)
#df
    #print(f'got an item: {item}')
    #print(f'Object name: {item.ObjectName}')
    #print(item._methods_)
    
#     if item.ObjectName =='AcDbRotatedDimension':
#         #print('found aligned dimention.')
#         #print(f'Object layer name: {item.Layer}\n')       
#         dim_rotated = item
#     elif item.ObjectName == 'AcDbAlignedDimension':
#         #print('found rotated dimention\n')
#         dim_aligned = item
        
    #explode blocks
#    elif item.ObjectName == 'AcDbBlockReference':       
#         try:
#             print(f'Found a block.\n')
#             block = item
#             block.Explode()
#         except:
#             pass    # here we have some VARIANT bug. So just proceed.
#         i+=1

Block name: <POINTER(IAcadBlock) ptr=0x771ab78 at c11be48>
Block name: <POINTER(IAcadBlock) ptr=0x4a3ce58 at c11b848>
Block name: <POINTER(IAcadBlock) ptr=0x72c0548 at c11be48>


In [117]:
for device in df['device'].unique():
    print(df)
    #if line is horizontal
    df['horizontal']= (df.iloc[:,1] == df.iloc[:,2])
    df['point_X'] = df.iloc[:, 1] if df['horizontal'] else (df.iloc[:, 1] - 10) 
    print(df)
    #df.iloc
    
    break

    device  start_att_line_X  end_att_line_X  start_att_line_Y  \
0        0        468.643810      473.643810        569.910725   
1        0        471.143810      471.143810        572.410725   
2        0        538.643810      543.643810        569.910725   
3        0        541.143810      541.143810        572.410725   
4        0        468.643810      473.643810        539.910725   
5        0        471.143810      471.143810        542.410725   
6        0        538.643810      543.643810        539.910725   
7        0        541.143810      541.143810        542.410725   
8        1        656.799864      661.799864        539.910725   
9        1        656.799864      661.799864        569.910725   
10       2        475.100512      475.100512        462.608155   
11       2        545.100512      545.100512        462.608155   

    end_att_line_Y  horizontal  
0       569.910725       False  
1       567.410725        True  
2       569.910725       False  
3       5

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [61]:
def FindAttachmentPoint(data):
    """This function converts given data to the coordinates of attachment points. 
    
    data: dataframe with coordinates of lines in layer 'attachment point'
    
    return
    """
    #FindAttachmentPoint(df[df['device']==device])
    print(f'data:\n{data}')
    print(data.iloc[[0]])


def MakeDimVertical(msp, point1, point2, loc):
    """This function make vertical dimension.
    
    msp:  model space
    point1, point2: points of dimention
    loc - location of dimension
    """
    pass
    

def MakeDimHorizontal(msp, point1, point2, loc):
    """This function make horizontal dimension.
    
    msp:  model space
    point1, point2: points of dimention
    loc - location of dimension
    """
    pass

def ProcessAttachmentPoints(msp, objects):
    """This function make coorginates of attachment points.
    
    msp:  model space
    objects:  lines from specified layer
    """
    pass


In [7]:
pt = array.array('d', [0,0,0]) # to convert in variant
point = mSp.AddPoint(pt)
# Add a LINE
pt1 = array.array('d', [0.0,0.0,0]) # start point
pt2 = array.array('d', [20.0,20.0,0]) # end point
line = mSp.AddLine(pt1, pt2)

NameError: name 'mSp' is not defined

In [ ]:
s1 =  array.array('d', [500.0, 800.0, 0.0])
s2 =  array.array('d', [500.0, 1200.0, 0.0])
loc = array.array('d', [500.0, 1000.0, 0.0])
new_dim = mSp.AddDimAligned(s1, s2, loc)
new_dim.update()

In [ ]:
acad = win32com.client.Dispatch("AutoCAD.Application")

doc = acad.ActiveDocument   # Document object


# iterate trough all objects (entities) in the currently opened drawing
# and if its a BlockReference, display its attributes and some other things.
for entity in acad.ActiveDocument.ModelSpace:
    name = entity.EntityName
    if name == 'AcDbBlockReference':
        HasAttributes = entity.HasAttributes
        if HasAttributes:
            print(entity.Name)
            print(entity.Layer)
            print(entity.ObjectID)
            for attrib in entity.GetAttributes():
                print("  {}: {}".format(attrib.TagString, attrib.TextString))
                
                # update text
                attrib.TextString = 'modified with python'
                attribib.Update()

In [ ]:
dim.


In [40]:
df = pd.read_excel('E://Python/RubinProject/Journal.xlsx')

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

In [ ]:
make_dim_vertical()

In [ ]:
df = df.rename({'Имя прибора': 'prib_name',
               'Индекс по схеме': 'shema_index'},
               axis=1)

Фичи размеров в качестве таргетов:
- TextPosition (rotated размеры тоже их имеют)

Запасной вариант- взрывать кхерам все размеры, вытаскивать параметры линий, и таким образом получать координаты точек привязки размера, а заодно и координаты текста. Наверное, придётся так и сделать, чтобы обеспечить непересекаемоть размеров и других линий чертежа, типа линий габаритки

todo:

точка крепления может быть не посередине линии соответствующего слоя, а где-то на пересечении с линией прибора. Сделать проверку на небольшое отклонение и выдать сообщение чтобы поправили чертёж